In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
# 📌 Step 1: Load and prepare data
df = pd.read_csv("/kaggle/input/dataset-initiale/job_data_cleaned_final.csv")

# Parse and clean skills
df["skill_list"] = df["Skills"].apply(lambda x: [s.strip().lower() for s in str(x).split(',')] if pd.notnull(x) else [])
df.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Job Title,Description,Location,Date,Company,Salary,URL,Skills,skill_list
0,Data Analyst,NaN,Serbia,25-09-2023,Cryptology,NaN,NaN,"excel, power bi, python, sql, tableau","[excel, power bi, python, sql, tableau]"
1,Data Analyst,NaN,United States,02-03-2023,Point32Health,NaN,http://tuftshealthplan.com/,"excel, sas, sql","[excel, sas, sql]"
2,Data Analyst,NaN,United States,26-04-2023,Apex Systems,NaN,NaN,"azure, databricks, jira, oracle, power bi, pyt...","[azure, databricks, jira, oracle, power bi, py..."
3,Data Engineer,NaN,Canada,25-05-2023,ODAIA Intelligence Inc.,NaN,http://www.odaia.ai/,"aws, flow, notion, python, sql, word","[aws, flow, notion, python, sql, word]"
4,Data Engineer,NaN,Germany,26-05-2023,DAHMEN Personalservice GmbH,NaN,NaN,"java, python, sql","[java, python, sql]"


In [4]:
# Count non-null skill lists
non_null_count = df['Skills'].notna().sum()

print(f"Number of rows with non-null skills: {non_null_count}")
print(f"Total rows in dataset: {len(df)}")
print(f"Percentage with skills: {non_null_count/len(df)*100:.2f}%")

Number of rows with non-null skills: 670364
Total rows in dataset: 787686
Percentage with skills: 85.11%


In [2]:
from sklearn.preprocessing import MultiLabelBinarizer
#  Multi-hot encoding
mlb = MultiLabelBinarizer()
X  = mlb.fit_transform(df["skill_list"])
skills = mlb.classes_
len(skills)

252

In [3]:
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


2025-05-19 19:54:59.222852: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747684499.737238      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747684499.878487      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Num GPUs Available: 2


In [4]:
import numpy as np

def generate_input_output(X, hide_fraction=0.3):
    X_input = X.copy()
    for i in range(X.shape[0]):
        idx = np.where(X[i] == 1)[0]
        hide_n = int(len(idx) * hide_fraction)
        hide_idx = np.random.choice(idx, hide_n, replace=False)
        X_input[i, hide_idx] = 0  # hide some known skills
    return X_input, X  # input = partial, output = full

X_input, y = generate_input_output(X)


In [5]:
from sklearn.model_selection import train_test_split
X_input_train, X_input_test, y_train, y_test = train_test_split(X_input, y, test_size=0.2, random_state=42)


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential([
    Dense(512, input_shape=(252,), activation='relu'),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(252, activation='sigmoid')  # one for each skill
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1747684533.218178      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1747684533.218972      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 512)                 │         129,536 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 252)                 │          64,764 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 325,628 (1.24 MB)

 Trainable params: 325,628 (1.24 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
model.fit(X_input_train, y_train, epochs=100, batch_size=512, validation_split=0.1)


Epoch 1/100


I0000 00:00:1747684561.430823     101 service.cc:148] XLA service 0x7d3994005c80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747684561.432169     101 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1747684561.432189     101 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1747684561.776900     101 cuda_dnn.cc:529] Loaded cuDNN version 90300


  63/1108 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0518 - loss: 0.4208

I0000 00:00:1747684563.821615     101 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1108/1108 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.1705 - loss: 0.0916 - val_accuracy: 0.2808 - val_loss: 0.0160
Epoch 2/100
1108/1108 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.2756 - loss: 0.0186 - val_accuracy: 0.2631 - val_loss: 0.0137
Epoch 3/100
1108/1108 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.2671 - loss: 0.0166 - val_accuracy: 0.2401 - val_loss: 0.0131
Epoch 4/100
1108/1108 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.2478 - loss: 0.0159 - val_accuracy: 0.2218 - val_loss: 0.0128
Epoch 5/100
1108/1108 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.2412 - loss: 0.0154 - val_accuracy: 0.2199 - val_loss: 0.0126
Epoch 6/100
1108/1108 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.2378 - loss: 0.0151 - val_accuracy: 0.2207 - val_loss: 0.0125
Epoch 7/100
1108/1108 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.2384 - loss: 0.0149 - val_accuracy: 0.2229 - val_loss: 0.0124
Epoch 8/100
1108/1108 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.2427 - loss: 0.0147 - val

In [8]:
from sklearn.metrics import precision_score, recall_score, f1_score
loss, accuracy = model.evaluate(X_input_test, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

y_pred = model.predict(X_input_test)
y_pred_binary = (y_pred > 0.5).astype(int)  # seuil de 0.5

# Métriques globales
print("Precision:", precision_score(y_test, y_pred_binary, average='micro'))
print("Recall:", recall_score(y_test, y_pred_binary, average='micro'))
print("F1-score:", f1_score(y_test, y_pred_binary, average='micro'))


4924/4924 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.4194 - loss: 0.0117
Test Loss: 0.0117, Test Accuracy: 0.4194
4924/4924 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step
Precision: 0.9796836165180889
Recall: 0.8359569007183214
F1-score: 0.9021315309657649


In [9]:
def recommend_skills(input_skills, top_k=5):
    input_vec = mlb.transform([input_skills])
    preds = model.predict(input_vec)[0]
    
    # Mask already known skills
    preds[input_vec[0] == 1] = 0
    top_indices = preds.argsort()[-top_k:][::-1]
    
    return [skills[i] for i in top_indices]

# Example
recommend_skills(['python', 'sql'])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step


['r', 'excel', 'azure', 'aws', 'tableau']

In [11]:
import joblib
model.save("skill_recommender.h5")
joblib.dump(mlb, "skill_label_binarizer.pkl")


['skill_label_binarizer.pkl']